# 1. Install and Import Dependencies

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate the Model

In [8]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# 3. Encode and calculate Sentiment

In [22]:
tokens = tokenizer.encode('i loved this cellphone, but it came with some scratches',return_tensors='pt')

In [23]:
print(tokens)
print(tokenizer.decode(tokens[0]))

tensor([[  101,   151, 46747, 10372, 16931, 34735,   117, 10502, 10197, 13430,
         10171, 10970, 72732, 10165,   102]])
[CLS] i loved this cellphone, but it came with some scratches [SEP]


In [24]:
result = model(tokens)

In [25]:
result.logits

tensor([[-2.4181, -0.7065,  1.5270,  1.7220, -0.2319]],
       grad_fn=<AddmmBackward0>)

In [26]:
print(int(torch.argmax(result.logits)) + 1,'Estrela')

4 Estrela


a sequencia numerica presente no tensor retornado representa a pontuacao pra cada sentimento. do menor para o maior (1,2,3,4,5 estrelas, por exemplo)

# 4. Collect Quotes

In [42]:
url = 'https://quotes.toscrape.com/'

In [49]:
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*quote.*')
results = soup.find_all('div',{'class':regex})
quotes = [result.text for result in results]

In [51]:
type(quotes)

list

In [73]:
def preprocess(texto):
  return re.search(r'“(.*?)”', texto).group(1)

In [74]:
formatted_quotes = []
for quote in quotes:
  formatted_quote = preprocess(quote)
  formatted_quotes.append(formatted_quote)

In [76]:
formatted_quotes

['The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.',
 'It is our choices, Harry, that show what we truly are, far more than our abilities.',
 'There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.',
 'The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.',
 "Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.",
 'Try not to become a man of success. Rather become a man of value.',
 'It is better to be hated for what you are than to be loved for what you are not.',
 "I have not failed. I've just found 10,000 ways that won't work.",
 "A woman is like a tea bag; you never know how strong it is until it's in hot water.",
 'A day without sunshine is like, you know, night.']

In [104]:
import pandas as pd

df = pd.DataFrame(formatted_quotes, columns=['quotes'])

In [105]:
df.head()

,quotes
0,The world as we have created it is a process o...
1,"It is our choices, Harry, that show what we tr..."
2,There are only two ways to live your life. One...
3,"The person, be it gentleman or lady, who has n..."
4,"Imperfection is beauty, madness is genius and ..."


In [147]:
def sentiment_analyzer(df,coluna):
  texts = list(df[coluna])
  pontuacoes = []

  for text in texts:
    tokens = tokenizer.encode(text,return_tensors='pt')
    result = model(tokens)
    pontuacao = (int(torch.argmax(result.logits)) + 1)
    pontuacoes.append([pontuacao])

  df['pontuacao'] = pontuacoes
  pontuacao_formatada = []

  for i,pontuacao in enumerate(list(df_analisado['pontuacao'])):
   pontuacao_formatada.append(pontuacao[0])

  df.drop('pontuacao',axis=1)
  df['pontuacao'] = pontuacao_formatada

  return df

In [148]:
df_analisado = sentiment_analyzer(df,'quotes')

In [149]:
df_analisado

,quotes,pontuacao
0,The world as we have created it is a process o...,5
1,"It is our choices, Harry, that show what we tr...",5
2,There are only two ways to live your life. One...,4
3,"The person, be it gentleman or lady, who has n...",1
4,"Imperfection is beauty, madness is genius and ...",1
5,Try not to become a man of success. Rather bec...,3
6,It is better to be hated for what you are than...,3
7,"I have not failed. I've just found 10,000 ways...",1
8,A woman is like a tea bag; you never know how ...,3
9,"A day without sunshine is like, you know, night.",5
